# Measure intensities with regionprops

In [ ]:
# Unregister => larger images
# unaligned => not turned to major axis
# 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from aicsimageprocessing import diagnostic_sheet, read_ome_zarr, rescale_image, imgtoprojection
from upath import UPath as Path
import warnings
from aicsimageio import transforms, AICSImage
import nbvv
import os
import numpy as np
import time
from scipy.spatial import distance
from serotiny.io.image import image_loader
# df = pd.read_parquet("s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet")
#df = pd.read_parquet("s3://allencell-hipsc-cytodata/hackathon_manifest_17oct2022.parquet")
#print(f'Number of cells: {len(df)}')
#print(f'Number of columns: {len(df.columns)}')

19-Oct-22 22:01:01 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [28]:
from skimage.measure import regionprops, regionprops_table
def get_regionprops_meas(img_data, cell_id):
    # Not meaningful, because of the rescaling
    # Can we calculate this on raw pixel values? => Guy is looking into this
    desired_properties = [
        "area",
        "centroid",
        "major_axis_length",
        "minor_axis_length",
        "extent",
        "equivalent_diameter",
    ]
    
    intensity_properties = [
        'intensity_mean',
        'intensity_max',
        'intensity_min',
    ]
    
    props = pd.DataFrame(regionprops_table(img_data[5, :, :, :], properties=desired_properties))
    props['CellId'] = cell_id
    
    props_int_bf = pd.DataFrame(regionprops_table(img_data[5, :, :, :], img_data[0, :, :, :], properties=intensity_properties)).add_prefix('bf_')
    
    props_int_dna = pd.DataFrame(regionprops_table(img_data[5, :, :, :], img_data[1, :, :, :], properties=intensity_properties)).add_prefix('dna_')

    props_int_membrane = pd.DataFrame(regionprops_table(img_data[5, :, :, :], img_data[2, :, :, :], properties=intensity_properties)).add_prefix('membrane_')

    props_int_structure = pd.DataFrame(regionprops_table(img_data[5, :, :, :], img_data[3, :, :, :], properties=intensity_properties)).add_prefix('structure_')
    
    return pd.concat([props, props_int_bf, props_int_dna, props_int_membrane, props_int_structure], axis=1)

In [3]:
df = pd.read_csv('/home/aicsuser/cytodata-hackathon-base/data/mitocells.csv')

In [4]:
df["raw_image"] = df["3d_image"].str.replace("register", "merge")
df["raw_image"] = df["raw_image"].str.replace("zarr", "tiff")

In [14]:
some_cell = df.sample(1).iloc[0]
# full_img = read_ome_zarr(some_cell["3d_image"], level = 2)
#full_img = read_ome_zarr(some_cell["raw_image"])
full_img = image_loader(some_cell["raw_image"])
full_img.shape
#img_data = full_img.data.squeeze()
#img_data.shape

/opt/conda/lib/python3.10/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


(7, 81, 536, 435)

In [ ]:
feature_df = pd.DataFrame()
for index, curr_cell in df.iterrows():
    print(index)
    # Get raw image data based on current cell ID
    full_img = image_loader(curr_cell["raw_image"])

    # Calculate regionprops based on image data
    curr_df = get_regionprops_meas(full_img, curr_cell['CellId'])

    # Add df to the main df
    feature_df = pd.concat([feature_df, curr_df])
    
df = pd.merge(df, feature_df, on='CellId')

In [ ]:
df